In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set() # matplot lib defaults
plt.rcParams['figure.figsize'] = (16, 8)
%config InlineBackend.figure_format='retina'

In [4]:
# find the notebook the saved figures came from
fig_prefix='../figures/jq-'

In [5]:
# from IPython.display import IFrame
# IFrame('https://dataviz.vam.wfp.org/economic_explorer/price-forecasts-alerts?adm0=205', width=700, height=400)

In [6]:
df =pd.read_csv('../data/eac-ratin.csv')
df.tail(5)

,Market,Product,Country,Date,Retail (MT),Wholesale (MT),Currency
70441,Tororo,Maize,Uganda,6/24/2019,1493160,1194527,UGX
70442,Tororo,Maize,Uganda,6/25/2019,1500000,1200000,UGX
70443,Tororo,Maize,Uganda,6/26/2019,1499998,1200001,UGX
70444,Tororo,Maize,Uganda,6/27/2019,1505092,1204076,UGX
70445,Tororo,Maize,Uganda,6/28/2019,1345949,987031,UGX


In [7]:
# import qgrid
# qgrid.nbinstall(overwrite=True)
# qgrid.show_grid(df, remote_js=True)

Data cleaning: NaN, dtype conversion

In [8]:
df = df.rename(columns = lambda x: x.lower())
cols = df.columns.tolist()
df = df.rename(columns={cols[-3]: 'retail', cols[-2]: 'wholesale'})
df.columns.tolist()

['market', 'product', 'country', 'date', 'retail', 'wholesale', 'currency']

In [9]:
df.dtypes

market       object
product      object
country      object
date         object
retail       object
wholesale    object
currency     object
dtype: object

In [10]:
df['wholesale'].sort_values().unique()

array(['0', '1', '1000', ..., '999998', '999999', 'Wholesale'],
      dtype=object)

In [11]:
df[df['wholesale']=='Wholesale'] = np.NaN
df['wholesale']=df['wholesale'].astype('float')

In [12]:
df['retail'].sort_values().unique()

array(['0', '1', '1000', ..., '999999', '9999998', nan], dtype=object)

In [13]:
df[df['retail']=='NaN'] = np.NaN
df['retail'] = df['retail'].astype('float')
df['retail'].sort_values().unique()

array([0.0000000e+00, 1.0000000e+00, 1.6000000e+02, ..., 1.0191057e+07,
       1.7999999e+07,           nan])

In [14]:
df['date'] = pd.to_datetime(df['date'])

In [15]:
df.dtypes

market               object
product              object
country              object
date         datetime64[ns]
retail              float64
wholesale           float64
currency             object
dtype: object

In [16]:
str_cols = ['market', 'product', 'country', 'currency']
df[str_cols].describe()

for item in str_cols:
    df[item]=df[item].astype('category') # which will by default set the length to the max len it encounters
    print(df[item].unique())

[Bujumbura, Gitega, Kobero market, Ngozi, NaN, ..., Masindi, Mbale, Owino, Soroti, Tororo]
Length: 59
Categories (58, object): [Bujumbura, Gitega, Kobero market, Ngozi, ..., Mbale, Owino, Soroti, Tororo]
[Maize, NaN]
Categories (1, object): [Maize]
[Burundi, NaN, DRC, Kenya, Rwanda, South Sudan, Tanzania, Uganda]
Categories (7, object): [Burundi, DRC, Kenya, Rwanda, South Sudan, Tanzania, Uganda]
[BIF, NaN, CDF, KES, RWF, SSD, TZS, UGX]
Categories (7, object): [BIF, CDF, KES, RWF, SSD, TZS, UGX]


In [17]:
df.dtypes

market             category
product            category
country            category
date         datetime64[ns]
retail              float64
wholesale           float64
currency           category
dtype: object

In [18]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df['week']=df['date'].dt.week
# note that since there are nan in the date colunms, let the dtype remain float

In [19]:
df.head(3)

,market,product,country,date,retail,wholesale,currency,year,month,day,week
0,Bujumbura,Maize,Burundi,2011-10-04,0.0,548798.0,BIF,2011.0,10.0,4.0,40.0
1,Bujumbura,Maize,Burundi,2011-10-05,0.0,549000.0,BIF,2011.0,10.0,5.0,40.0
2,Bujumbura,Maize,Burundi,2011-10-06,0.0,498970.0,BIF,2011.0,10.0,6.0,40.0


In [20]:
np.nan


nan

In [21]:
# replace zeros with NaN

cols = ['wholesale', 'retail']
df[cols] = df[cols].replace({0:np.nan})

# cond = (df['wholesale']==0)
# df_zero = df[cond]
# df_zero['wholesale'] = np.NaN
# df['wholesale']=df_zero['wholesale']


In [22]:
# return 1 to confirm no zero values
np.prod(df['wholesale']!=0)

1

In [23]:
# remove outliers:
def remove_outliers(x):
    '''x: pandas series'''
    lower_bound, upper_bound = x.quantile(.05), x.quantile(.95)
    x = x[x.between(lower_bound, upper_bound)]
    return x

df['wholesale'] = remove_outliers(df['wholesale'])
df['retail'] = remove_outliers(df['retail'])

exploratory visualization:

In [24]:
# save a copy of cleaned data to csv
df.to_csv('../data/cleaned_data.csv', index=False)

In [25]:
# save to db file
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/mydb.db', echo=False)
df.to_sql('data', con=engine, if_exists='replace',
           index_label='id')